## 로그인 및 모델 관련 정보 명시


In [ ]:
from huggingface_hub import login

HF_TOKEN = "" # 실행 전 토큰 입력
RANDOM_STATE = 42
BASE_MODEL_ID = "klue/roberta-base"
TUNED_MODEL_PATH = "./koRoBERTa_base-checkpoint16000"

login(HF_TOKEN)

c:\Users\user\anaconda3\envs\ocean\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 모델과 토크나이저 생성

In [2]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, AutoConfig

# 모델 설정 로드
config = AutoConfig.from_pretrained(
    f"{TUNED_MODEL_PATH}/config.json",
    local_files_only=True
)

# 시퀀스 분류 모델로 변경 (5개 OCEAN 점수 예측)
# 원본 RoBERTa를 시퀀스 분류 모델로 변경
config.num_labels = 5  # OCEAN 5가지 차원
config.problem_type = "regression"  # 회귀 문제로 설정

# 시퀀스 분류 모델 로드
model = RobertaForSequenceClassification.from_pretrained( # 회귀에도 사용됨
    TUNED_MODEL_PATH,
    config=config,
    local_files_only=True
)
 
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token # padding 명시

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./koRoBERTa_base-checkpoint16000 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 데이터셋 불러오기

In [ ]:
import pandas as pd
from datasets import Dataset

csv_file_path = "./dataset/instruction_tuning_data.csv"
df = pd.read_csv(csv_file_path)

dataset_hf = Dataset.from_pandas(df)
print(dataset_hf)

Dataset({
    features: ['text', 'OCEAN'],
    num_rows: 11307
})


## OCEAN -> 숫자 배열

In [4]:
def parse_ocean_scores(ocean_str):
    try:
        clean_str = ocean_str.strip('[]').strip()
        if ',' in clean_str:
            scores = [float(x.strip()) for x in clean_str.split(',')]
        else:
            scores = [float(x) for x in clean_str.split()]
        
        # 정확히 5개 점수인지 확인
        if len(scores) != 5:
            print(f"경고: 예상치 못한 OCEAN 점수 형식: {ocean_str}, 결과: {scores}")
            # 부족한 경우 0으로 채움
            while len(scores) < 5:
                scores.append(0.0)
            # 넘치는 경우 자름
            scores = scores[:5]
        
        return scores
    except Exception as e:
        print(f"OCEAN 점수 파싱 오류: {e}, 입력: {ocean_str}")
        return [0.0, 0.0, 0.0, 0.0, 0.0]  # 오류 시 기본값 반환


## 데이터셋 토큰화

In [5]:
def tokenize_and_prepare(examples):
    result = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors=None  # batch 처리를 위해 None으로 설정
    )
    
    labels = []
    for ocean_str in examples["OCEAN"]:
        scores = parse_ocean_scores(ocean_str)
        labels.append(scores)
    
    result["labels"] = labels
    
    return result

## 데이터셋에 변환 함수 적용 및 스플릿

In [6]:
processed_dataset = dataset_hf.map(
    tokenize_and_prepare,
    batched=True,
    remove_columns=dataset_hf.column_names
)

train_test_split = processed_dataset.train_test_split(test_size=0.2, seed=RANDOM_STATE)
train_dataset = train_test_split['train']
temp_dataset = train_test_split['test']

valid_test_split = temp_dataset.train_test_split(test_size=0.5, seed=RANDOM_STATE)
eval_dataset = valid_test_split['train']
test_dataset = valid_test_split['test']

Map: 100%|██████████| 11307/11307 [00:02<00:00, 5574.88 examples/s]


In [7]:
print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"검증 데이터셋 크기: {len(eval_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")

학습 데이터셋 크기: 9045
검증 데이터셋 크기: 1131
테스트 데이터셋 크기: 1131


## 모델 학습

In [8]:
from peft import (
    LoraConfig,
    get_peft_model,
)
import numpy as np
from transformers import  TrainingArguments, Trainer

# 모델을 LoRA 적용 전에 학습 가능한 상태로 설정
for param in model.classifier.parameters():
    param.requires_grad = True
 
lora_config = LoraConfig(
    r=32,
    target_modules=["query", "key", "value", "output.dense"],
    lora_dropout=0.05,
    task_type="SEQ_CLS", # 시퀀스 레벨 회귀도 처리함
    bias="none",
)

peft_model = get_peft_model(model, lora_config)

print(f"학습 가능한 파라미터: {peft_model.print_trainable_parameters()}")

# 학습 설정
training_args = TrainingArguments(
    output_dir="./outputs_ocean_regression",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1, 
    num_train_epochs=100,  
    learning_rate=2e-4,
    fp16=True,
    logging_strategy="epoch", 
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy_like",
    greater_is_better=True,
    optim="adamw_torch",
    report_to="tensorboard",
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mse = np.mean((predictions - labels) ** 2)
    mae = np.mean(np.abs(predictions - labels))
    accuracy_like = 1 - mae
    dimension_mae = np.mean(np.abs(predictions - labels), axis=0)
    dimension_acc = 1 - dimension_mae
    
    return {
        "mse": mse,
        "mae": mae,
        "accuracy_like": accuracy_like,
        "openness_acc": dimension_acc[0],
        "conscientiousness_acc": dimension_acc[1],
        "extraversion_acc": dimension_acc[2], 
        "agreeableness_acc": dimension_acc[3], 
        "neuroticism_acc": dimension_acc[4],
        "openness_mae": dimension_mae[0],
        "conscientiousness_mae": dimension_mae[1],
        "extraversion_mae": dimension_mae[2],
        "agreeableness_mae": dimension_mae[3],
        "neuroticism_mae": dimension_mae[4]
    }

# Trainer 초기화
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# 학습 실행
print("학습 시작...")
trainer.train()

# 모델 저장
trainer.save_model("./koRoBERT_OCEAN")
print("학습 완료 및 모델 저장됨!")

# 테스트 데이터셋 평가
print("테스트 데이터셋 평가...")
test_results = trainer.evaluate(test_dataset)
print(f"테스트 전체 정확도(1-MAE): {test_results['eval_accuracy_like']:.4f}")
print("\n각 OCEAN 차원별 정확도(1-MAE):")
print(f"개방성(Openness): {test_results['eval_openness_acc']:.4f}")
print(f"성실성(Conscientiousness): {test_results['eval_conscientiousness_acc']:.4f}")
print(f"외향성(Extraversion): {test_results['eval_extraversion_acc']:.4f}")
print(f"친화성(Agreeableness): {test_results['eval_agreeableness_acc']:.4f}")
print(f"신경성(Neuroticism): {test_results['eval_neuroticism_acc']:.4f}")

trainable params: 4,428,293 || all params: 115,050,250 || trainable%: 3.8490
학습 가능한 파라미터: None


c:\Users\user\anaconda3\envs\ocean\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_31624\1729033331.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


학습 시작...


  1%|          | 2261/226100 [12:30<22:31:27,  2.76it/s]

{'loss': 0.0599, 'grad_norm': 3.405261278152466, 'learning_rate': 2e-05, 'epoch': 1.0}


...

{'loss': 0.0026, 'grad_norm': 1.0482231378555298, 'learning_rate': 9.828492800629024e-08, 'epoch': 99.99}


                                                          
100%|██████████| 226100/226100 [20:27:41<00:00,  3.07it/s]


{'eval_loss': 0.02950669266283512, 'eval_mse': 0.02950669638812542, 'eval_mae': 0.13268470764160156, 'eval_accuracy_like': 0.8673152923583984, 'eval_openness_acc': 0.8627369999885559, 'eval_conscientiousness_acc': 0.88225919008255, 'eval_extraversion_acc': 0.8641557097434998, 'eval_agreeableness_acc': 0.8741634488105774, 'eval_neuroticism_acc': 0.8532613515853882, 'eval_openness_mae': 0.1372630000114441, 'eval_conscientiousness_mae': 0.11774082481861115, 'eval_extraversion_mae': 0.13584430515766144, 'eval_agreeableness_mae': 0.1258365362882614, 'eval_neuroticism_mae': 0.1467386782169342, 'eval_runtime': 6.2784, 'eval_samples_per_second': 180.142, 'eval_steps_per_second': 22.617, 'epoch': 99.99}
{'train_runtime': 73661.1116, 'train_samples_per_second': 12.279, 'train_steps_per_second': 3.069, 'train_loss': 0.008629677516071736, 'epoch': 99.99}
학습 완료 및 모델 저장됨!
테스트 데이터셋 평가...


100%|██████████| 142/142 [00:06<00:00, 22.66it/s]

테스트 전체 정확도(1-MAE): 0.8761

각 OCEAN 차원별 정확도(1-MAE):
개방성(Openness): 0.8716
성실성(Conscientiousness): 0.8875
외향성(Extraversion): 0.8717
친화성(Agreeableness): 0.8813
신경성(Neuroticism): 0.8683


## 추론

In [9]:
# test_dataset을 이용한 OCEAN 점수 예측 및 평가 코드
import torch
import numpy as np
from tqdm.auto import tqdm

def inspect_dataset_structure(dataset):
    """
    데이터셋의 구조를 검사하고 출력하는 함수
    
    Args:
        dataset: 검사할 데이터셋
    """
    print("데이터셋 크기:", len(dataset))
    print("데이터셋 특성:", dataset.features)
    
    # 첫 번째 샘플 확인
    first_sample = dataset[0]
    print("\n첫 번째 샘플 구조:")
    for key, value in first_sample.items():
        if isinstance(value, (torch.Tensor, np.ndarray)):
            print(f"{key}: 타입={type(value)}, 형태={value.shape}")
        else:
            print(f"{key}: 타입={type(value)}")
    
    return first_sample

def predict_ocean_simple(dataset, model):
    """
    간소화된 예측 함수 - 한 번에 전체 데이터셋을 처리하지 않고 샘플별로 처리
    
    Args:
        dataset: 예측에 사용할 데이터셋
        model: 학습된 모델
        
    Returns:
        predictions: 모든 샘플의 예측 결과 (numpy 배열)
        labels: 모든 샘플의 실제 레이블 (numpy 배열)
    """
    # 결과 저장용 리스트 초기화
    all_predictions = []
    all_labels = []
    
    # 모델을 평가 모드로 설정
    model.eval()
    
    # 데이터셋 샘플별로 처리
    for i in tqdm(range(len(dataset)), desc="Predicting"):
        sample = dataset[i]
        
        # 입력 데이터 추출
        # 데이터셋 구조에 따라 수정 필요할 수 있음
        try:
            # 데이터를 텐서로 변환하고 차원 추가
            input_ids = torch.tensor(sample['input_ids']).unsqueeze(0)
            attention_mask = torch.tensor(sample['attention_mask']).unsqueeze(0)
            
            # GPU 사용 가능한 경우 GPU로 이동
            if torch.cuda.is_available():
                input_ids = input_ids.to("cuda")
                attention_mask = attention_mask.to("cuda")
                
            # 그래디언트 계산 없이 추론 수행
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                
            # 예측 결과 가져오기
            predictions = outputs.logits.cpu().numpy()[0]  # 배치 차원 제거
            all_predictions.append(predictions)
            
            # 레이블 가져오기
            if 'labels' in sample:
                all_labels.append(sample['labels'])
                
        except Exception as e:
            print(f"샘플 {i} 처리 중 오류 발생: {e}")
            print(f"샘플 내용: {sample}")
            continue
    
    # 결과를 배열로 변환
    all_predictions = np.array(all_predictions)
    
    # 레이블이 있는 경우에만 처리
    if all_labels:
        all_labels = np.array(all_labels)
    else:
        all_labels = None
    
    return all_predictions, all_labels

def evaluate_predictions(predictions, labels):
    """
    예측 결과를 평가하는 함수
    
    Args:
        predictions: 모델의 예측값 (numpy 배열)
        labels: 실제 레이블 (numpy 배열)
        
    Returns:
        metrics: 다양한 평가 지표를 포함한 딕셔너리
    """
    # 레이블이 없는 경우 평가 불가
    if labels is None:
        print("경고: 레이블이 없어 평가를 수행할 수 없습니다.")
        return None
    
    # 예측값을 0-1 범위로 클리핑
    clipped_predictions = np.clip(predictions, 0.0, 1.0)
    
    # 전체 MSE, MAE 계산
    mse = np.mean((clipped_predictions - labels) ** 2)
    mae = np.mean(np.abs(clipped_predictions - labels))
    accuracy_like = 1 - mae  # 1-MAE 계산 (정확도와 유사한 지표)
    
    # 각 OCEAN 차원별 지표 계산
    dimension_mse = np.mean((clipped_predictions - labels) ** 2, axis=0)
    dimension_mae = np.mean(np.abs(clipped_predictions - labels), axis=0)
    dimension_acc = 1 - dimension_mae  # 차원별 1-MAE
    
    # OCEAN 차원 이름
    dimensions = ["개방성(Openness)", "성실성(Conscientiousness)", 
                  "외향성(Extraversion)", "친화성(Agreeableness)", 
                  "신경성(Neuroticism)"]
    
    # 평가 지표 결과 저장
    metrics = {
        "overall_mse": mse,
        "overall_mae": mae,
        "overall_accuracy": accuracy_like,
    }
    
    # 차원별 지표 추가
    for i, dim in enumerate(dimensions):
        metrics[f"{dim}_mse"] = dimension_mse[i]
        metrics[f"{dim}_mae"] = dimension_mae[i]
        metrics[f"{dim}_accuracy"] = dimension_acc[i]
    
    return metrics

def display_sample_predictions(predictions, labels=None, num_samples=5):
    """
    샘플 예측 결과를 표시하는 함수
    
    Args:
        predictions: 모델의 예측값 (numpy 배열)
        labels: 실제 레이블 (numpy 배열), 없을 수 있음
        num_samples: 표시할 샘플 수
    """
    # 표시할 샘플 수 결정 (데이터셋 크기보다 크지 않게)
    num_samples = min(num_samples, len(predictions))
    
    # OCEAN 차원 이름
    dimensions = ["개방성(Openness)", "성실성(Conscientiousness)", 
                  "외향성(Extraversion)", "친화성(Agreeableness)", 
                  "신경성(Neuroticism)"]
    
    # 지정된 샘플 수만큼 결과 표시
    for i in range(num_samples):
        print(f"\n샘플 #{i+1}")
        
        # 예측 결과 표시
        print("예측 OCEAN 점수:")
        for j, dim in enumerate(dimensions):
            pred_score = np.clip(predictions[i][j], 0.0, 1.0)
            if labels is not None:
                true_score = labels[i][j]
                diff = abs(pred_score - true_score)
                print(f"  {dim}: {pred_score:.4f} (실제: {true_score:.4f}, 오차: {diff:.4f})")
            else:
                print(f"  {dim}: {pred_score:.4f}")

# 메인 실행 코드
def main():
    """
    테스트 데이터셋을 이용한 OCEAN 점수 예측 및 평가를 실행하는 메인 함수
    """
    # 먼저 데이터셋 구조 확인
    print("데이터셋 구조 확인 중...")
    first_sample = inspect_dataset_structure(test_dataset)
    
    print("\n테스트 데이터셋에서 OCEAN 점수 예측 중...")
    
    try:
        # 간소화된 샘플별 예측 방식 시도
        predictions, labels = predict_ocean_simple(
            test_dataset, 
            peft_model
        )
        
        # 예측 결과 평가 (레이블이 있는 경우)
        if labels is not None:
            print("\n테스트 데이터셋 평가 결과:")
            metrics = evaluate_predictions(predictions, labels)
            
            # 전체 성능 지표 출력
            print(f"전체 MSE: {metrics['overall_mse']:.4f}")
            print(f"전체 MAE: {metrics['overall_mae']:.4f}")
            print(f"전체 정확도(1-MAE): {metrics['overall_accuracy']:.4f}")
            
            # 차원별 성능 지표 출력
            print("\n차원별 정확도(1-MAE):")
            dimensions = ["개방성(Openness)", "성실성(Conscientiousness)", 
                        "외향성(Extraversion)", "친화성(Agreeableness)", 
                        "신경성(Neuroticism)"]
            
            for dim in dimensions:
                print(f"{dim}: {metrics[f'{dim}_accuracy']:.4f}")
        
        # 샘플 예측 결과 표시
        print("\n샘플 예측 결과:")
        display_sample_predictions(predictions, labels, num_samples=5)
        
        # 결과를 파일로 저장 (선택 사항)
        print("\n결과를 파일로 저장 중...")
        np.save("ocean_predictions.npy", predictions)
        if labels is not None:
            np.save("ocean_labels.npy", labels)
        
    except Exception as e:
        print(f"예측 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    
    print("평가 완료!")

# 스크립트가 직접 실행될 때만 main 함수 호출
if __name__ == "__main__":
    main()

데이터셋 구조 확인 중...
데이터셋 크기: 1131
데이터셋 특성: {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

첫 번째 샘플 구조:
input_ids: 타입=<class 'list'>
token_type_ids: 타입=<class 'list'>
attention_mask: 타입=<class 'list'>
labels: 타입=<class 'list'>

테스트 데이터셋에서 OCEAN 점수 예측 중...


Predicting: 100%|██████████| 1131/1131 [00:36<00:00, 30.65it/s]


테스트 데이터셋 평가 결과:
전체 MSE: 0.0224
전체 MAE: 0.1239
전체 정확도(1-MAE): 0.8761

차원별 정확도(1-MAE):
개방성(Openness): 0.8716
성실성(Conscientiousness): 0.8875
외향성(Extraversion): 0.8717
친화성(Agreeableness): 0.8813
신경성(Neuroticism): 0.8683

샘플 예측 결과:

샘플 #1
예측 OCEAN 점수:
  개방성(Openness): 0.6431 (실제: 0.5000, 오차: 0.1431)
  성실성(Conscientiousness): 0.6587 (실제: 0.5625, 오차: 0.0962)
  외향성(Extraversion): 0.6807 (실제: 0.6875, 오차: 0.0068)
  친화성(Agreeableness): 0.5977 (실제: 0.5625, 오차: 0.0352)
  신경성(Neuroticism): 0.4207 (실제: 0.2708, 오차: 0.1499)

샘플 #2
예측 OCEAN 점수:
  개방성(Openness): 0.6216 (실제: 0.7500, 오차: 0.1284)
  성실성(Conscientiousness): 0.6660 (실제: 0.5833, 오차: 0.0827)
  외향성(Extraversion): 0.6758 (실제: 0.6667, 오차: 0.0091)
  친화성(Agreeableness): 0.5972 (실제: 0.5208, 오차: 0.0764)
  신경성(Neuroticism): 0.4175 (실제: 0.2917, 오차: 0.1258)

샘플 #3
예측 OCEAN 점수:
  개방성(Openness): 0.5835 (실제: 0.3125, 오차: 0.2710)
  성실성(Conscientiousness): 0.6235 (실제: 0.7500, 오차: 0.1265)
  외향성(Extraversion): 0.6152 (실제: 0.5625, 오차: 0.0527)
  친화성(Agreeableness)